# Analyzing FEMA's National Flood Insurance Program (NFIP) Data With DuckDB
Author: Mark Bauer

# Introduction

The FEMA National Flood Insurance Program (NFIP) website offers a trove of valuable information. Among its highlights is the [Flood Insurance Data and Analytics](https://nfipservices.floodsmart.gov/reports-flood-insurance-data) section, featuring data visualizations, tables, and reports. This project was inspired by these resources, particularly the Financial Losses by State and Policy, and Loss Statistics by Flood Zone Excel files.

Additionally, this analysis focuses solely on the [NFIP Redated Claims](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2) dataset, and fortunately, it is available as a Parquet file. Given the size of this dataset, it was a good opportunity to utilize and learn more about [DuckDB](https://duckdb.org/).

# Data
Title: **OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2**  

You can find information about the data, as well as the data dictionary, here: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2.

In [1]:
# import libraries
import duckdb
import pandas as pd

In [2]:
print("duckdb version: {}".format(duckdb.__version__))

duckdb version: 0.10.0


In [3]:
# create a DuckDB database instance
con = duckdb.connect(config={"allow_unsigned_extensions": "true"})

# assign the path of the httpfs extension. Note: I built the extension from source but you
# should review the extensions guide on DuckDB's official documentation.
path = "../duckdb/build/release/repository/v0.10.0/osx_amd64/httpfs.duckdb_extension"

# install and load extension
con.install_extension(path)
con.load_extension(path)

# Create Table Using DuckDB

In [5]:
# create table of the claims dataset
con.sql(
    """
    CREATE TABLE claims AS
    FROM read_parquet('data/FimaNfipClaims.parquet')
    """
)

# examine count of rows
con.sql("SELECT COUNT(*) AS count_rows FROM claims").show()

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│    2609973 │
└────────────┘



# Examine and Describe Data Set

In [6]:
# preview data
con.sql("SELECT * FROM claims LIMIT 10").show()

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬──────────┬───────────┐
│          id          │ agricultureStructu…  │       asOfDate       │ … │ censusBlockGroupFips │ latitude │ longitude │
│       varchar        │       boolean        │      timestamp       │   │       varchar        │  double  │  double   │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼──────────┼───────────┤
│ 61a2811b-a92b-486d…  │ false                │ 2020-12-11 16:25:4…  │ … │ 010030114072         │     30.3 │     -87.7 │
│ b9123f27-fa0e-4097…  │ false                │ 2020-11-13 14:50:3…  │ … │ 010030114073         │     30.3 │     -87.7 │
│ 9de3700b-bbbd-408c…  │ false                │ 2022-04-21 19:56:4…  │ … │ 010010204001         │     32.5 │     -86.4 │
│ c2450563-0de0-40df…  │ false                │ 2020-03-09 19:28:2…  │ … │ 010010204001         │     32.5 │     -86.4 │
│ 6b13ea94-ef8d-428a…  │ false  

In [7]:
# examine column metadata and assign as a pandas df
describe_df = con.sql("DESCRIBE claims").df()

describe_df

,column_name,column_type,null,key,default,extra
0,id,VARCHAR,YES,None,None,None
1,agricultureStructureIndicator,BOOLEAN,YES,None,None,None
2,asOfDate,TIMESTAMP,YES,None,None,None
3,basementEnclosureCrawlspaceType,SMALLINT,YES,None,None,None
4,policyCount,SMALLINT,YES,None,None,None
...,...,...,...,...,...,...
68,countyCode,VARCHAR,YES,None,None,None
69,censusTract,VARCHAR,YES,None,None,None
70,censusBlockGroupFips,VARCHAR,YES,None,None,None
71,latitude,DOUBLE,YES,None,None,None


In [8]:
# examine each column in sections because of large number of columns
describe_df.iloc[:25, :]

,column_name,column_type,null,key,default,extra
0,id,VARCHAR,YES,None,None,None
1,agricultureStructureIndicator,BOOLEAN,YES,None,None,None
2,asOfDate,TIMESTAMP,YES,None,None,None
3,basementEnclosureCrawlspaceType,SMALLINT,YES,None,None,None
4,policyCount,SMALLINT,YES,None,None,None
5,crsClassificationCode,SMALLINT,YES,None,None,None
6,dateOfLoss,DATE,YES,None,None,None
7,elevatedBuildingIndicator,BOOLEAN,YES,None,None,None
8,elevationCertificateIndicator,VARCHAR,YES,None,None,None
9,elevationDifference,DOUBLE,YES,None,None,None


In [9]:
describe_df.iloc[25:50, :]

,column_name,column_type,null,key,default,extra
25,postFIRMConstructionIndicator,BOOLEAN,YES,None,None,None
26,rateMethod,VARCHAR,YES,None,None,None
27,smallBusinessIndicatorBuilding,BOOLEAN,YES,None,None,None
28,totalBuildingInsuranceCoverage,BIGINT,YES,None,None,None
29,totalContentsInsuranceCoverage,BIGINT,YES,None,None,None
30,yearOfLoss,SMALLINT,YES,None,None,None
31,primaryResidenceIndicator,BOOLEAN,YES,None,None,None
32,buildingDamageAmount,BIGINT,YES,None,None,None
33,buildingDeductibleCode,VARCHAR,YES,None,None,None
34,netBuildingPaymentAmount,DOUBLE,YES,None,None,None


In [10]:
describe_df.iloc[50:, :]

,column_name,column_type,null,key,default,extra
50,netIccPaymentAmount,DOUBLE,YES,None,None,None
51,nfipRatedCommunityNumber,VARCHAR,YES,None,None,None
52,nfipCommunityNumberCurrent,VARCHAR,YES,None,None,None
53,nfipCommunityName,VARCHAR,YES,None,None,None
54,nonPaymentReasonContents,VARCHAR,YES,None,None,None
55,nonPaymentReasonBuilding,VARCHAR,YES,None,None,None
56,numberOfUnits,INTEGER,YES,None,None,None
57,buildingReplacementCost,BIGINT,YES,None,None,None
58,contentsReplacementCost,BIGINT,YES,None,None,None
59,replacementCostBasis,VARCHAR,YES,None,None,None


# Summary Statistics

In [11]:
# calculate summary statistics of each column
summarize_df = con.sql("SUMMARIZE claims").df()

summarize_df

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,000006af-51d1-453e-8c06-5791060175ab,fffffeef-b65f-4405-ab4e-60257523aa4f,2662059,None,None,None,None,None,2609973,0.00
1,agricultureStructureIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2609973,0.00
2,asOfDate,TIMESTAMP,2019-09-19 06:12:43.388,2024-05-01 20:18:36.77,425022,None,None,None,None,None,2609973,0.00
3,basementEnclosureCrawlspaceType,SMALLINT,0,4,4,1.228374267022299,1.0566934718476721,0,1,2,2609973,70.18
4,policyCount,SMALLINT,1,1090,398,1.2797094069555508,6.67351428857187,1,1,1,2609973,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
68,countyCode,VARCHAR,01001,78030,2942,None,None,None,None,None,2609973,2.38
69,censusTract,VARCHAR,01001020100,78030961200,54666,None,None,None,None,None,2609973,5.27
70,censusBlockGroupFips,VARCHAR,010010201001,780309612002,110152,None,None,None,None,None,2609973,5.27
71,latitude,DOUBLE,-36.0,69.9,335,33.86958959935434,5.826147648469343,29.885555420103564,31.034309863474476,39.741329849029,2609973,1.53


In [12]:
# examine each column in sections because of large number of columns
summarize_df.iloc[:25, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,000006af-51d1-453e-8c06-5791060175ab,fffffeef-b65f-4405-ab4e-60257523aa4f,2662059,None,None,None,None,None,2609973,0.00
1,agricultureStructureIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2609973,0.00
2,asOfDate,TIMESTAMP,2019-09-19 06:12:43.388,2024-05-01 20:18:36.77,425022,None,None,None,None,None,2609973,0.00
3,basementEnclosureCrawlspaceType,SMALLINT,0,4,4,1.228374267022299,1.0566934718476721,0,1,2,2609973,70.18
4,policyCount,SMALLINT,1,1090,398,1.2797094069555508,6.67351428857187,1,1,1,2609973,0.00
5,crsClassificationCode,SMALLINT,1,10,10,6.848428755453079,1.5049489596950096,5,7,8,2609973,82.30
6,dateOfLoss,DATE,1978-01-01,2024-05-01,16687,None,None,None,None,None,2609973,0.00
7,elevatedBuildingIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2609973,0.00
8,elevationCertificateIndicator,VARCHAR,1,E,9,None,None,None,None,None,2609973,77.41
9,elevationDifference,DOUBLE,-9989.0,998.0,370,1.2817296900477881,29.362656199635108,0.0,1.0,2.7745562409888866,2609973,72.86


In [13]:
summarize_df.iloc[25:50, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
25,postFIRMConstructionIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2609973,0.00
26,rateMethod,VARCHAR,1,W,22,None,None,None,None,None,2609973,1.90
27,smallBusinessIndicatorBuilding,BOOLEAN,false,true,2,None,None,None,None,None,2609973,0.00
28,totalBuildingInsuranceCoverage,BIGINT,0,243903000,11951,166889.41585940993,1228597.5907276564,38979,99733,209278,2609973,0.00
29,totalContentsInsuranceCoverage,BIGINT,0,6000000,3048,30733.689877477143,50621.54742463355,0,11481,44532,2609973,0.00
30,yearOfLoss,SMALLINT,1978,2024,47,2002.1435083811211,12.501478526204501,1993,2005,2012,2609973,0.00
31,primaryResidenceIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2609973,0.00
32,buildingDamageAmount,BIGINT,0,927700000,201765,36311.10027419284,814576.3551107972,3442,11247,40081,2609973,22.40
33,buildingDeductibleCode,VARCHAR,0,H,15,None,None,None,None,None,2609973,11.99
34,netBuildingPaymentAmount,DOUBLE,-162432.16,10000000.0,1280807,24695.035330748662,59271.29895574326,0.15364584889707653,4723.460869598978,24764.392949312303,2609973,0.00


In [14]:
summarize_df.iloc[50:, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
50,netIccPaymentAmount,DOUBLE,-6450.0,60000.0,8708,361.25060566526923,3098.8927849140314,0.0,0.0,0.0,2609973,0.00
51,nfipRatedCommunityNumber,VARCHAR,000000,999999,16288,None,None,None,None,None,2609973,0.00
52,nfipCommunityNumberCurrent,VARCHAR,0000,815000,11987,None,None,None,None,None,2609973,72.96
53,nfipCommunityName,VARCHAR,ABBEVILLE COUNTY *,"ZUMBRO FALLS, CITY OF",9794,None,None,None,None,None,2609973,72.43
54,nonPaymentReasonContents,VARCHAR,01,99,23,None,None,None,None,None,2609973,68.43
55,nonPaymentReasonBuilding,VARCHAR,01,99,23,None,None,None,None,None,2609973,77.62
56,numberOfUnits,INTEGER,0,99999,445,1.3626484479940102,66.17856487595769,1,1,1,2609973,0.11
57,buildingReplacementCost,BIGINT,0,9999000000,443562,8613520.156327939,235672086.90846258,0,120306,223120,2609973,22.40
58,contentsReplacementCost,BIGINT,0,20000000,8867,2802.1066546353063,46646.036696565956,0,0,0,2609973,59.11
59,replacementCostBasis,VARCHAR,A,R,2,None,None,None,None,None,2609973,7.27


In [15]:
# examine columns with high null percentage
(summarize_df
 .sort_values(by='null_percentage', ascending=False)
 .loc[:, ['column_name', 'null_percentage']]
 .head(10)
)

,column_name,null_percentage
45,floodCharacteristicsIndicator,98.47
43,eventDesignationNumber,96.01
5,crsClassificationCode,82.30
14,lowestAdjacentGrade,81.38
55,nonPaymentReasonBuilding,77.62
8,elevationCertificateIndicator,77.41
15,lowestFloorElevation,76.24
10,baseFloodElevation,75.68
62,floodZoneCurrent,73.00
52,nfipCommunityNumberCurrent,72.96


## Amount Paid on Claims Summary Statistics

In [16]:
# examine summary statisitics on amount claims paid columns
col_names = [
    'amountPaidOnBuildingClaim',
    'amountPaidOnContentsClaim',
    'amountPaidOnIncreasedCostOfComplianceClaim'
]

cols = [
    'column_name', 'min', 'max', 'avg', 'std',
    'q25', 'q50', 'q75', 'null_percentage'
]

(summarize_df
 .loc[summarize_df['column_name'].isin(col_names), cols]
 .astype({'avg':'float', 'std':'float', 'q25':'float', 'q50':'float', 'q75':'float'})
 .round({'avg':0, 'std':0, 'q25':0, 'q50':0, 'q75':0})
 .set_index('column_name')
 .reindex(col_names)
 .T
)

column_name,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim
min,-162432.16,-41276.32,-6450.0
max,10000000.0,757048.95,60000.0
avg,31499.0,7115.0,462.0
std,65354.0,22203.0,3498.0
q25,2325.0,0.0,0.0
q50,9640.0,0.0,0.0
q75,36545.0,4839.0,0.0
null_percentage,21.5,21.5,21.5


## Building Damage, Values, and Replacement Costs Summary Statistics

In [17]:
# examine summary statisitics on building damage amounts, property values and replacement costs columns
col_names = [
    'buildingDamageAmount',
    'netBuildingPaymentAmount',
    'buildingPropertyValue',
    'buildingReplacementCost'
    
]

cols = [
    'column_name', 'min', 'max', 'avg', 'std',
    'q25', 'q50', 'q75', 'null_percentage'
]

(summarize_df
 .loc[summarize_df['column_name'].isin(col_names), cols]
 .astype({'avg':'float', 'std':'float', 'q25':'float', 'q50':'float', 'q75':'float'})
 .round({'avg':0, 'std':0, 'q25':0, 'q50':0, 'q75':0})
 .set_index('column_name')
 .reindex(col_names)
 .T
)

column_name,buildingDamageAmount,netBuildingPaymentAmount,buildingPropertyValue,buildingReplacementCost
min,0,-162432.16,0,0
max,927700000,10000000.0,9950000000,9999000000
avg,36311.0,24695.0,7185179.0,8613520.0
std,814576.0,59271.0,200484955.0,235672087.0
q25,3442.0,0.0,60707.0,0.0
q50,11247.0,4723.0,109667.0,120306.0
q75,40081.0,24764.0,189103.0,223120.0
null_percentage,22.4,0.0,22.4,22.4


## Contents Damage, Values, and Replacement Costs Summary Statistics

In [18]:
# examine summary statisitics on contents amounts, property values and replacement costs columns
col_names = [
    'contentsDamageAmount',
    'netContentsPaymentAmount',
    'contentsPropertyValue',
    'contentsReplacementCost'
    
]

cols = [
    'column_name', 'min', 'max', 'avg', 'std',
    'q25', 'q50', 'q75', 'null_percentage'
]

(summarize_df
 .loc[summarize_df['column_name'].isin(col_names), cols]
 .astype({'avg':'float', 'std':'float', 'q25':'float', 'q50':'float', 'q75':'float'})
 .round({'avg':0, 'std':0, 'q25':0, 'q50':0, 'q75':0})
 .set_index('column_name')
 .reindex(col_names)
 .T
)

column_name,contentsDamageAmount,netContentsPaymentAmount,contentsPropertyValue,contentsReplacementCost
min,0,-41276.32,-1,0
max,19230507,757048.95,281895323,20000000
avg,18360.0,5563.0,31188.0,2802.0
std,85635.0,19864.0,488976.0,46646.0
q25,1634.0,0.0,0.0,0.0
q50,5724.0,0.0,0.0,0.0
q75,18585.0,2229.0,19516.0,0.0
null_percentage,59.08,0.0,59.08,59.11


## Elevation and Water Depth Summary Statistics

In [19]:
# examine summary statisitics on elevation and water depth columns
col_names = [
    'lowestAdjacentGrade',
    'lowestFloorElevation',
    'baseFloodElevation',
    'elevationDifference',    
    'waterDepth'
    
]

cols = [
    'column_name', 'min', 'max', 'avg', 'std',
    'q25', 'q50', 'q75', 'null_percentage'
]

(summarize_df
 .loc[summarize_df['column_name'].isin(col_names), cols]
 .astype({'avg':'float', 'std':'float', 'q25':'float', 'q50':'float', 'q75':'float'})
 .round({'avg':0, 'std':0, 'q25':0, 'q50':0, 'q75':0})
 .set_index('column_name')
 .reindex(col_names)
 .T
)

column_name,lowestAdjacentGrade,lowestFloorElevation,baseFloodElevation,elevationDifference,waterDepth
min,-99999.9,-6002.0,-9999.0,-9989.0,-999
max,9998.9,9998.0,9998.0,998.0,999
avg,46.0,118.0,124.0,1.0,4.0
std,1570.0,716.0,751.0,29.0,16.0
q25,3.0,6.0,7.0,0.0,0.0
q50,6.0,10.0,9.0,1.0,1.0
q75,11.0,17.0,14.0,3.0,2.0
null_percentage,81.38,76.24,75.68,72.86,8.5


# Analysis

In [20]:
# preview column names to use for Analysis section
(con
 .sql("SELECT * FROM claims LIMIT 10")
 .df()
 .info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 73 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   id                                          10 non-null     object        
 1   agricultureStructureIndicator               10 non-null     bool          
 2   asOfDate                                    10 non-null     datetime64[ns]
 3   basementEnclosureCrawlspaceType             3 non-null      float64       
 4   policyCount                                 10 non-null     int16         
 5   crsClassificationCode                       7 non-null      float64       
 6   dateOfLoss                                  10 non-null     datetime64[ns]
 7   elevatedBuildingIndicator                   10 non-null     bool          
 8   elevationCertificateIndicator               1 non-null      object        
 9   elevationDiff

## Total Claims

In [21]:
con.sql(
    """
    SELECT
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    """
)

┌─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│    int64    │     int64     │     int64     │             int64              │
├─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ 80057600349 │   64534521973 │   14576678596 │                      946399781 │
└─────────────┴───────────────┴───────────────┴────────────────────────────────┘

## Total Claims by State

In [22]:
con.sql(
    """
    SELECT
        state,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
    """
)

┌─────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│  state  │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│ varchar │    int64    │     int64     │     int64     │             int64              │
├─────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ LA      │ 20838923032 │   16431635833 │    4129930893 │                      277356306 │
│ TX      │ 17083042210 │   13136650534 │    3888589851 │                       57801825 │
│ FL      │ 11166770151 │    9458190896 │    1671373622 │                       37205633 │
│ NJ      │  6465642998 │    5345477364 │     869804214 │                      250361420 │
│ NY      │  5714558618 │    4944383614 │     717516208 │                       52658796 │
│ MS      │  3121617364 │    2385247785 │     675466706 │                       60902873 │
│ NC      │  2002988094 │    1692155613 │     273870198 │                       36962283 │

## Total Claims by State for Fiscal Year 2024

In [23]:
con.sql(
    """
    SELECT
        state, 
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    WHERE dateOfLoss between '2023-10-01' AND '2024-09-30'
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
    """
)

┌─────────┬────────────┬───────────────┬───────────────┬────────────────────────────────┐
│  state  │ TotalClaim │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│ varchar │   int64    │     int64     │     int64     │             int64              │
├─────────┼────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ ME      │   28553103 │      26108035 │       2435049 │                          10019 │
│ NY      │   23722092 │      21998415 │       1723677 │                              0 │
│ FL      │   23229815 │      21388638 │       1841177 │                              0 │
│ CA      │   21744654 │      20373883 │       1370772 │                              0 │
│ NJ      │   19296081 │      17150204 │       2145877 │                              0 │
│ RI      │    8546350 │       8129389 │        416960 │                              0 │
│ SC      │    8444517 │       7837273 │        607244 │                              0 │
│ NH      

## Total Claims by County

In [24]:
con.sql(
    """
    SELECT
        state,
        countyCode,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 20
    """
)

┌─────────┬────────────┬────────────┬───────────────┬───────────────┬────────────────────────────────┐
│  state  │ countyCode │ TotalClaim │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│ varchar │  varchar   │   int64    │     int64     │     int64     │             int64              │
├─────────┼────────────┼────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ TX      │ 48201      │ 8705456456 │    6756752295 │    1928687883 │                       20016278 │
│ LA      │ 22071      │ 7294446959 │    5890983538 │    1292969906 │                      110493515 │
│ LA      │ 22051      │ 3527668438 │    2619027797 │     847127476 │                       61513166 │
│ FL      │ 12071      │ 3333790855 │    2975824827 │     356154969 │                        1811059 │
│ NJ      │ 34029      │ 2607850866 │    2160793243 │     275246922 │                      171810701 │
│ TX      │ 48167      │ 2456431694 │    1887282122 │     548178802 │    

## Total Claims by Flood Event

In [25]:
con.sql(
    """
    SELECT
        floodEvent,
        yearOfLoss,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS ICCClaim
    FROM claims
    WHERE floodEvent NOT NULL
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 20
    """
)

┌──────────────────────────┬────────────┬─────────────┬───────────────┬───────────────┬───────────┐
│        floodEvent        │ yearOfLoss │ TotalClaim  │ BuildingClaim │ ContentsClaim │ ICCClaim  │
│         varchar          │   int16    │    int64    │     int64     │     int64     │   int64   │
├──────────────────────────┼────────────┼─────────────┼───────────────┼───────────────┼───────────┤
│ Hurricane Katrina        │       2005 │ 16261697056 │   12659081935 │    3360020221 │ 242594900 │
│ Hurricane Harvey         │       2017 │  9055446500 │    6925357028 │    2115077279 │  15012193 │
│ Hurricane Sandy          │       2012 │  8956175518 │    7707763973 │     951584858 │ 296826687 │
│ Hurricane Ian            │       2022 │  4653622925 │    4132266664 │     519501586 │   1854675 │
│ Hurricane Ike            │       2008 │  2702511916 │    2073801567 │     577791589 │  50918760 │
│ Mid-summer severe storms │       2016 │  2533368883 │    2175216439 │     347263588 │  10888856 │


In [26]:
con.sql(
    """
    SELECT
        floodEvent,
        yearOfLoss,
        state,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS ICCClaim
    FROM claims
    WHERE floodEvent NOT NULL
    GROUP BY 1, 2, 3
    ORDER BY 4 DESC
    LIMIT 20
    """
)

┌──────────────────────────┬────────────┬─────────┬─────────────┬───────────────┬───────────────┬───────────┐
│        floodEvent        │ yearOfLoss │  state  │ TotalClaim  │ BuildingClaim │ ContentsClaim │ ICCClaim  │
│         varchar          │   int16    │ varchar │    int64    │     int64     │     int64     │   int64   │
├──────────────────────────┼────────────┼─────────┼─────────────┼───────────────┼───────────────┼───────────┤
│ Hurricane Katrina        │       2005 │ LA      │ 13347164644 │   10441743931 │    2721451049 │ 183969663 │
│ Hurricane Harvey         │       2017 │ TX      │  9040104864 │    6912536831 │    2112555840 │  15012193 │
│ Hurricane Ian            │       2022 │ FL      │  4608058052 │    4090976598 │     515346779 │   1734675 │
│ Hurricane Sandy          │       2012 │ NJ      │  4372117786 │    3673312870 │     457640033 │ 241164883 │
│ Hurricane Sandy          │       2012 │ NY      │  4213915671 │    3702363706 │     465358598 │  46193367 │
│ Mid-summ

## Total Claims by Flood Zone
*Formerly called floodZone. NFIP Flood Zone derived from the Flood Insurance Rate Map (FIRM) used to rate the insured property. A - Special Flood with no Base Flood Elevation on FIRM; AE, A1-A30 - Special Flood with Base Flood Elevation on FIRM; A99 - Special Flood with Protection Zone; AH, AHB* - Special Flood with Shallow Ponding; AO, AOB* - Special Flood with Sheet Flow; X, B - Moderate Flood from primary water source. Pockets of areas subject to drainage problems; X, C - Minimal Flood from primary water source. Pockets of areas subject to drainage problems; D - Possible Flood; V - Velocity Flood with no Base Flood Elevation on FIRM; VE, V1-V30 - Velocity Flood with Base Flood Elevation on FIRM; AE, VE, X - New zone designations used on new maps starting January 1, 1986, in lieu of A1-A30, V1-V30, and B and C; AR - A Special Flood Hazard Area that results from the decertification of a previously accredited flood protection system that is determined to be in the process of being restored to provide base flood protection; AR Dual Zones - (AR/AE, AR/A1-A30, AR/AH, AR/AO, AR/A) Areas subject to flooding from failure of the flood protection system (Zone AR) which also overlap an existing Special Flood Hazard Area as a dual zone; *AHB, AOB, ARE, ARH, ARO, and ARA are not risk zones shown on a map, but are acceptable values for rating purposes*

In [27]:
con.sql(
    """
    SELECT
        ratedFloodZone as ratedFloodZone,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
    """
)

┌────────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ ratedFloodZone │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│    varchar     │    int64    │     int64     │     int64     │             int64              │
├────────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ AE             │ 35111157725 │   29248455727 │    5349491157 │                      513210841 │
│ X              │ 13293684798 │   10258338304 │    3006893246 │                       28453247 │
│ B              │  3826003058 │    2835227048 │     971988985 │                       18787025 │
│ A              │  3097345563 │    2457942866 │     602004885 │                       37397812 │
│ C              │  2851078461 │    2117465674 │     723537437 │                       10075351 │
│ A04            │  2185900631 │    1800894098 │     364396806 │                       20609726 │
│ A03            │  

## Total Claims by Occupancy Type
*Code indicating the use and occupancy type of the insured structure. Note, 2-digit codes are for Risk Rating 2.0 policies. 1=single family residence; 2 = 2 to 4 unit residential building; 3 = residential building with more than 4 units; 4 = Non-residential building; 6 = Non Residential - Business; 11 = Single-family residential building with the exception of a mobile home or a single residential unit within a multi unit building; 12 = A residential non-condo building with 2, 3, or 4 units seeking insurance on all units; 13 = A residential non-condo building with 5 or more units seeking insurance on all units; 14 = Residential mobile/manufactured home; 15 = Residential condo association seeking coverage on a building with one or more units; 16 = Single residential unit within a multi-unit building; 17 = Non-residential mobile/manufactured home; 18 = A non-residential building; 19 = a non-residential unit within a multi-unit building;*

In [28]:
con.sql(
    """
    SELECT
        occupancyType,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
    """
)

┌───────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ occupancyType │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│     int16     │    int64    │     int64     │     int64     │             int64              │
├───────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│             1 │ 58068945128 │   46435050309 │   10743686447 │                      890208372 │
│             4 │  6399065992 │    4292302635 │    2095965716 │                       10797641 │
│             3 │  4374189293 │    4228633027 │     138249314 │                        7306952 │
│             2 │  3730087287 │    3407331381 │     287664029 │                       35091878 │
│             6 │  3358494979 │    2511223292 │     845669820 │                        1601866 │
│            11 │  2491648290 │    2158598189 │     331902381 │                        1147721 │
│            15 │   861271847 

## Total Claims by Flood Zone and Occupancy Type

In [29]:
con.sql(
    """
    SELECT
        ratedFloodZone,
        occupancyType,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 20
    """
)

┌────────────────┬───────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ ratedFloodZone │ occupancyType │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│    varchar     │     int16     │    int64    │     int64     │     int64     │             int64              │
├────────────────┼───────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ AE             │             1 │ 24280260327 │   19986111836 │    3803579791 │                      490568700 │
│ X              │             1 │ 10707127507 │    8229644367 │    2450235094 │                       27248046 │
│ B              │             1 │  2959535823 │    2183555532 │     758744816 │                       17235475 │
│ A              │             1 │  2380346133 │    1945172272 │     399071426 │                       36102435 │
│ AE             │             3 │  2150998205 │    2096708194 │      52374043 │        

## Total Claims by Basement Enclosure Crawlspace Type
*Basement is defined for purposes of the NFIP as any level or story which has its floor subgrade on all sides. Basement structure values are as follows: 0 - None; 1 - Finished Basement/Enclosure; 2 - Unfinished Basement/Enclosure; 3 - Crawlspace; 4 - Subgrade Crawlspace;*

In [30]:
con.sql(
    """
    SELECT
        basementEnclosureCrawlspaceType,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
    """
)

┌─────────────────────────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ basementEnclosureCrawlspaceType │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│              int16              │    int64    │     int64     │     int64     │             int64              │
├─────────────────────────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│                            NULL │ 55401524778 │   43679431564 │   10971592499 │                      750500715 │
│                               0 │ 13914353649 │   11440401898 │    2409318309 │                       64633442 │
│                               2 │  6000512303 │    5288346590 │     619947436 │                       92218277 │
│                               1 │  3397191736 │    2939798687 │     439264817 │                       18128232 │
│                               4 │  1344017882 │    1186543234 │     136555535 

## Total Claims by Cause of Damage
*Indicates the method by which the insured's property and contents were damaged. Legal values (value : description): 0 : Other causes; 1 : Tidal water overflow; 2 : Stream, river, or lake overflow; 3 : Alluvial fan overflow; 4 : Accumulation of rainfall or snowmelt; 7 : Erosion-demolition; 8 : Erosion-removal; 9 : Earth movement, landslide, land subsidence, sinkholes, etc.; A : Closed basin lake; B : Expedited claim handling process without site inspection; C : Expedited claim handling process follow-up site inspection; D : Expedited claim handling process by Adjusting Process Pilot Program (Remote Adjustment); NOTE: Due to certain provisions of the Upton Jones Amendment to the National Flood Insurance Act, cause of loss codes 7 and 8 may be used only if the date of loss is prior to September 23, 1995. More than one cause of loss code can be selected. For example, you may select 2 (stream, river, or lake overflow) and D (remote adjustment) on a single claim, or any combination of letters/numbers as appropriate*

In [31]:
con.sql(
    """
    SELECT
        causeOfDamage,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
    """
)

┌───────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ causeOfDamage │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│    varchar    │    int64    │     int64     │     int64     │             int64              │
├───────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ 4             │ 31961230504 │   25625037178 │    6055472130 │                      280721196 │
│ 1             │ 21036248351 │   17233069923 │    3436391995 │                      366786433 │
│ 2             │ 18384564348 │   14699663364 │    3469240983 │                      215660001 │
│ 0             │  6142937074 │    4972163141 │    1102060910 │                       68713023 │
│ B             │  2101842424 │    1650532490 │     440000279 │                       11309655 │
│ 3             │   235661209 │     184005406 │      49416789 │                        2239014 │
│ NULL          │    52776078 

## Total Claims by Building Description
*Indicates the description of the use of the insured building. Legal values (value : description): 01 : Main House; 02 : Detached Guest House; 03 : Detached Garage; 04 : Agricultural Building; 05 : Warehouse; 06 : Pool House, Clubhouse, Recreation Building; 07 : Tool/Storage Shed; 08 : Other; 09 : Barn; 10 : Apartment Building; 11 : Apartment - Unit; 12 : Cooperative Building; 13 : Cooperative - Unit; 14 : Commercial Building; 15 : Condominium (Entire Building); 16 : Condominium - Unit; 17 : House of Worship; 18 : Manufactured (Mobile) Home; 19 : Travel Trailer; 20 : Townhouse/Rowhouse;*

In [32]:
con.sql(
    """
    SELECT
        buildingDescriptionCode,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
    """
)

┌─────────────────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ buildingDescriptionCode │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│          int16          │    int64    │     int64     │     int64     │             int64              │
├─────────────────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│                       1 │ 38774742941 │   32254938612 │    6101059138 │                      418745191 │
│                    NULL │ 38024521753 │   29673165989 │    7827229024 │                      524126740 │
│                       8 │  2710101630 │    2157722123 │     549626076 │                        2753431 │
│                       5 │   194328201 │     125807067 │      68521134 │                              0 │
│                      10 │   185624947 │     184269467 │       1355480 │                              0 │
│                       2 │    577967

## Total Claims by Rental Property Indicator
*Indicates if the property is a rental property. Yes is indicated with either a 'true' or '1'. No is indicated with either a 'false' or '0'. NOTE: A rental property is a property from which the owner receives payment from the occupant(s), known as tenants, in return for occupying or using the property. Rental properties may be either residential or commercial.*

In [33]:
con.sql(
    """
    SELECT
        rentalPropertyIndicator,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::INT64 AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::INT64 AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::INT64 AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
    """
)

┌─────────────────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ rentalPropertyIndicator │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│         boolean         │    int64    │     int64     │     int64     │             int64              │
├─────────────────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ false                   │ 78673563989 │   63312641722 │   14417199537 │                      943722731 │
│ true                    │  1384036360 │    1221880251 │     159479059 │                        2677050 │
└─────────────────────────┴─────────────┴───────────────┴───────────────┴────────────────────────────────┘